In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = "train.csv"
titanic = pd.read_csv(data)

titanic.head()

In [ ]:
titanic.info() # indicates we have missing values in 3 columns. 891 rows and 12 columns

In [ ]:
titanic.describe() # summary statistics for numerical columns 

In [ ]:
missing_values = titanic.isnull().sum() # Age - 177 missing values, Cabin - 687 missing values, Embarked - 2 missing values
missing_values[missing_values > 0] # displays columns with missing values 

In [ ]:
# Handling missing values 

# dropping the "Cabin" column

titanic = titanic.drop(columns=["Cabin"])

# dropping rows where "Embarked" is missing

titanic = titanic.dropna(subset=["Embarked"])

In [ ]:
# fill missing Age values with the median

titanic["Age"].fillna(titanic["Age"].median(), inplace=True)

In [ ]:
# looking at survival rate by gender 

titanic.groupby("Sex")[["Survived"]].mean() # 3 out every 4 females survived, 1 out of every 5 males survived

In [ ]:
# shows how class and gender affected the survival rate 

titanic.groupby(["Sex", "Pclass"])["Survived"].aggregate("mean").unstack()

In [ ]:
# The pivot table which was introduced for convinence can handle the multidimensional aggregation as seen above 

# The sex and the classes are actually good predictors of survival 
titanic.pivot_table("Survived", index="Sex", columns="Pclass") # makes the code a lot easier to read 

In [ ]:
# Multilevel pivot tables:
# Using the Age variable as a third dimension:

age = pd.cut(titanic["Age"], [0, 18, 80]) # bining the Age variable into 3 categories
titanic.pivot_table("Survived", ["Sex", age], "Pclass")

In [ ]:
fare = pd.qcut(titanic["Fare"], 2)
titanic.pivot_table("Survived", ["Sex", age, "Embarked"], "Pclass")

In [ ]:
titanic["Survived"].value_counts()

In [ ]:
titanic.groupby("Embarked")[["Survived"]].mean()

In [ ]:
titanic.pivot_table("Survived", index="Embarked", columns="Pclass")

In [ ]:
# Understanding the distribution of individual features and how they relate to the target variable 

# Analyzing Numerical features:

# Age: Age is an important factor as certain age groups (like children) may have been prioritized for survival 

# Plotting the Age distribution:

plt.figure(figsize=(10, 6))
sns.histplot(titanic["Age"], bins=30, kde=True) # kde overlays a smooth curve that helps to see the overall shape
plt.title("Age Distribution")
plt.xlabel("Age")
plt.show() # has a normal distribution

In [ ]:
# Boxplot to visualize the relationship between Age and Survived: there is likely to be more correlation with lesser age and survival

plt.figure(figsize=(10, 6))
sns.boxplot(x="Survived", y="Age", data=titanic, palette="Set2")
plt.title("Age vs Survived")
plt.xlabel("Survived")
plt.ylabel("Age")
plt.xticks([0, 1], ["Did Not Survive", "Survived"])
plt.show()

In [ ]:
# Fare: Higher prices could indicate wealthier passengers who may have had higher chances of survival

# Plotting the distribution for Fare:

plt.figure(figsize=(10, 6))
sns.histplot(titanic["Fare"], bins=30, kde=True)
plt.title("Fare Distribution")
plt.xlabel("Fare")
plt.show() # has a right-skewed distribution since most passengers likely paid lower fares, with fewer high-paying outliers

In [ ]:
# Boxplot for Fare and Survival: passengers with higher Fare prices has more chances of survival

plt.figure(figsize=(10, 6))
sns.boxplot(x="Survived", y="Fare", data=titanic, palette="coolwarm")
plt.title("Fare vs Survived")
plt.xlabel("Survived")
plt.ylabel("Fare")
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(titanic["SibSp"], kde=True)
plt.title("Siblings/Spouse distribution")
plt.xlabel("Sibsp")
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(titanic["Parch"], kde=True)
plt.title("Siblings/Spouse distribution")
plt.xlabel("Sibsp")
plt.show()

In [ ]:
# Analyzing Categorical Features: Pclass, Sex, Embarked

# Pclass: Passenger's class can indicate social status, which may correlate with survival rates

# Countplot for Pclass: Plots the survival rate for each class

# A countplot displays the count of survivors and non-survivors within each class, 
# making it easy to see how survival varied across classes.

plt.figure(figsize=(8, 5))
sns.countplot(x="Pclass", hue="Survived", data=titanic)
plt.title("Pclass vs Survival")
plt.xlabel("Pclass")
plt.ylabel("Count")
plt.legend(title="Survived")
plt.show() # a cool viz!. Indicates that the Pclass feature is a good predictor for survival tendencies.
# socio-economic status influenced survival

In [ ]:
# Sex: Gender may have affected survival as due to the "women and children first" policy

# Plotting survival rates by gender:

plt.figure(figsize=(8, 5))
sns.countplot(x="Sex", hue="Survived", data=titanic)
plt.title("Sex vs Survival")
plt.xlabel("Sex")
plt.ylabel("Count")
plt.legend(title="Survived")
plt.show() # the female had more survival tendencies than men

In [ ]:
# Embarked: The port of embarkation could influence survival, as different ports had different type of passengers

# Countplot for Embarkation:

plt.figure(figsize=(8, 5))
sns.countplot(x="Embarked", hue="Survived", data=titanic)
plt.title("Embarked vs Survival")
plt.xlabel("Embarked")
plt.ylabel("Count")
plt.legend(title="Survived")
plt.show()

### Feature Engineering

In [ ]:
# Creating new features:

# Creating an Age Group feature:

# define Age categories:

# The labels will allow for the analysis of survival by age category 
titanic["AgeGroup"] = pd.cut(titanic["Age"], bins=[0, 12, 18, 60, 100], labels=["Child", "Teen", "Adult", "Senior"])

In [ ]:
#  Family size, derived from SibSp (siblings/spouses) and Parch (parents/children), might indicate whether traveling in
# groups influenced survival.

# Creating a Family Size feature:

titanic["FamilySize"] = titanic["SibSp"] + titanic["Parch"] + 1 # plus 1 for the passenger 

In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(x="FamilySize", hue="Survived", data=titanic)
plt.title("Family Size vs Survival")
plt.xlabel("Family Size")
plt.legend(title="Survived")
plt.show()

In [ ]:
# Traveling alone versus with a group could have impacted survival chances, so it’s useful to flag passengers who had no 
# family with them.

# Creating Alone feature:

titanic["Alone"] = (titanic["FamilySize"] == 1).astype(int) # .astype(int) converts the result to integer value (0 or 1 )

### Encoding Categorical Variables 

In [ ]:
# Encoding Sex feature:

titanic["Sex"] = titanic["Sex"].map({"male": 0, "female": 1}) # the will help the model to recognize gender differences numerically

In [ ]:
# Encoding AgeGroup and Embarked:

# AgeGroup and Embarked are multi-class categorical variables, so we’ll use one-hot encoding to convert each category 
# into its own binary column.

titanic = pd.get_dummies(titanic, columns=["AgeGroup", "Embarked"], drop_first=True, dtype=int)
# pd.get_dummies converts each category into a new column
# drop_first=True removes one category to avoid multicolinearity (a redundancy issue where categories are perfectly correlated).

### Scaling Numerical Features

In [ ]:
# Scaling Fare:

# Fare has a wide range of values. Scaling it helps to bring it to a comparable range with other features.

from sklearn.preprocessing import StandardScaler

# Initializing the scaler:
scaler = StandardScaler() # normalizes the data to have a mean of 0 and std of 1, which helps the model to interpret 
# values consistently 

# Scaling the Fare feature:
titanic["Fare"] = scaler.fit_transform(titanic[["Fare"]]) # fit_transform learns and applys the scaling directly to Fare 

In [ ]:
# Final check for missing values:

titanic.isnull().sum()

In [ ]:
# Feature Type Refinement:

titanic.dtypes

In [ ]:
# Addressing Class Imbalance:

# Visualizing the Class Distribution:

sns.countplot(x="Survived", data=titanic, palette="viridis")
plt.title("Survival Class Distribution")
plt.xlabel("Survived")
plt.ylabel("Count")
plt.show()

#### Handling the Name feature

In [ ]:
# Extracting Titles from Name: This might help to reveal useful insights about social standing, age or even gender 

# We can extract titles (e.g., Mr, Mrs, Miss, Master) and use them as a new categorical feature 

titanic["Title"] = titanic["Name"].str.extract(" ([A-Za-z]+)\.", expand=False)

In [ ]:
titanic["Title"].value_counts()

In [ ]:
# Grouping rare titles: Reducing unique values simplifies the model’s task, especially if some titles only appear a few times in the dataset.

# Simplifying rare titles:

titanic["Title"] = titanic["Title"].replace(["Lady", "Countess", "Capt", "Col", "Don", "Dr", "Major", "Rev", "Sir", "Jonkheer", "Dona"], "Rare")
titanic["Title"] = titanic["Title"].replace(["Mlle", "Ms"], "Miss")
titanic["Title"] = titanic["Title"].replace("Mme", "Mrs")

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x="Title", hue="Survived", data=titanic)
plt.title("Title vs Survived")
plt.xlabel("Title")
plt.ylabel("Count")
plt.legend(title="Survived")
plt.show()

In [ ]:
# Encoding Title as Categorical Feature:

# One-hot encode Title feature:

titanic = pd.get_dummies(titanic, columns=["Title"], drop_first=True, dtype=int)

In [ ]:
# Dropping the original Name and Ticket columns:

titanic = titanic.drop(["Name", "Ticket"], axis=1)

In [ ]:
titanic.head() # we are left with numerical categorical variables

### Splitting the data

In [ ]:
# Define features (X) and target (y)

X = titanic.drop("Survived", axis=1)
y = titanic["Survived"]

In [ ]:
# Splitting into training and test sets:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Model Training 

In [ ]:
# Model Training with Logistic Regrssion:

#  A common choice for binary classification problems. It calculates probabilities and can work well with fewer features.

from sklearn.linear_model import LogisticRegression

# Initializing the Logistic Regression model
log_reg = LogisticRegression(random_state=42)

In [ ]:
# Fitting the model to the training data:

# Fitting (or training) the model allows it to learn patterns in the training data. The fit method updates the model’s parameters to minimize
# errors in predicting the target variable (Survived).

log_reg.fit(X_train, y_train)

### Evaluating the Model

In [ ]:
# Making Predictions:

# Generate predictions on the test set

y_pred = log_reg.predict(X_test)

In [ ]:
# Evaluating with Accuracy Score:

from sklearn.metrics import accuracy_score

# Calculating Accuracy:

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%") 

In [ ]:
# Evaluating with Classification Report:

# A classification report provides a deeper look, including precision, recall, and f1-score for each class, 
# helping us understand performance beyond accuracy.

from sklearn.metrics import classification_report

# Print classification report 

print(classification_report(y_test, y_pred))

### Testing Alternative Models

In [ ]:
# Implementing and Evaluating Each Model:

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Looping through each model, training it, and checking its accuracy on the test set:

# Dictionary of models:

models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42), 
    "Random Forest": RandomForestClassifier(random_state=42),
    "Support Vector Machine": SVC(random_state=42)
}

In [ ]:
models.items()

In [ ]:
# Evaluating each model:

for model_name, model in models.items():
    # Train the model:
    model.fit(X_train, y_train)

    # Make predictions:
    y_pred = model.predict(X_test)

    # Calculate accuracy:
    accuracy = accuracy_score(y_test, y_pred)

    print(f"{model_name} Accuracy: {accuracy * 100:.2f}%")

In [ ]:
print("Test set score: {:.2f}".format(log_reg.score(X_test, y_test))) # just checking...

### Hyperparameter Tuning for Logistic Regression

In [ ]:
# Using Grid Search for Hyperparmeter tuning: Grid Search is a method to exhaustively search over a set of specified hyperparameters to find the 
# best combination

# C - Controls regularization strength. Lower values mean stronger regularization strength.
# solver - Determines the optimization algorithm. Different solvers can have an impact on performance.
# penalty - Regularization type (l1, l2, etc.), which helps control model complexity and prevent overfitting.

from sklearn.model_selection import GridSearchCV

# Define parameter grid

param_grid = {
    "C": [0.02, 0.2, 2, 20, 200], 
    "solver": ["newton-cg", "lbfgs", "liblinear"],
    "penalty": ["l2"]
} 


# Initializing grid search:

grid_search = GridSearchCV(estimator=LogisticRegression(random_state=42), param_grid=param_grid, cv=5, scoring="accuracy", n_jobs=1)

# Perform grid search on training data

grid_search.fit(X_train, y_train)

In [ ]:
# Best parameters and Best score

print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_ * 100)

#### Training the Logistic Regression with the Best Parameters

In [ ]:
# Get best parameters from grid search 

best_params = grid_search.best_params_

# Train the model with best parameters

optimized_model = LogisticRegression(**best_params, random_state=42)
optimized_model.fit(X_train, y_train)

# Make predictions 

y_pred_optimized = optimized_model.predict(X_test)

# Calculate accuracy

accuracy_optimized = accuracy_score(y_test, y_pred_optimized)

print(f"Optimized Logistic Regression Accuracy: {accuracy_optimized * 100:.2f}%")

### Hyperparamter Tuning for Random Forest Classifier 

In [ ]:
# Define parameter grid for Random Forest 

param_grid_rf = {
    "n_estimators": [50, 100, 200],          # Number of trees
    "max_depth": [None, 10, 20, 30],         # Maximum depth of trees
    "min_samples_split": [2, 5, 10],         # Minimum samples to split a node
    "min_samples_leaf": [1, 2, 4],           # Minimum samples at leaf node
    "max_features": ["auto", "sqrt", "log2"] # Number of features to consider when looking for the best split
}

# Initialize GridSearchCV

grid_search_rf = GridSearchCV(estimator=RandomForestClassifier(random_state=42), param_grid=param_grid_rf, cv=5, scoring="accuracy", n_jobs=1)

In [ ]:
# Performing Grid Search:

# Training the Random Forest model with different combinations of hyperparameters and evaluating each one using 5-fold cross-validation.

# Fit grid search on training data

grid_search_rf.fit(X_train, y_train)

# Output the best parameters and the best cross-validation accuracy score

print("Best Parameters for Random Forest:", grid_search_rf.best_params_)
print("Best Cross-Validation Accuracy for Random Forest:", grid_search_rf.best_score_ * 100)

#### Training the Random Forest Model with Optimized Parameters

In [ ]:
# Get the best parameters from grid search 

best_params_rf = grid_search_rf.best_params_

# Train Random Forest with best parameters 

optimized_rf = RandomForestClassifier(**best_params_rf, random_state=42)
optimized_rf.fit(X_train, y_train)

# Making predictions 

y_pred_rf_optimized = optimized_rf.predict(X_test)

# Calculate accuracy

accuracy_rf_optimized = accuracy_score(y_test, y_pred_rf_optimized)
print(f"Optimized Random Forest Accuracy: {accuracy_rf_optimized * 100:.2f}%")

### Working on the Test Data 

In [ ]:
# Loading test data 

test_data = pd.read_csv("test.csv")

In [ ]:
missing_values = test_data.isnull().sum()
missing_values[missing_values > 0]

In [ ]:
test_data = test_data.drop(columns=["Cabin"])

In [ ]:
test_data["Age"].fillna(test_data["Age"].median(), inplace=True)

In [ ]:
test_data["Fare"].fillna(test_data["Fare"].median(), inplace=True)

In [ ]:
# Feature Engineering the test data

test_data["AgeGroup"] = pd.cut(test_data["Age"], bins=[0, 12, 18, 60, 100], labels=["Child", "Teen", "Adult", "Senior"])

In [ ]:
test_data["FamilySize"] = test_data["SibSp"] + test_data["Parch"] + 1


In [ ]:
test_data["Alone"] = (test_data["FamilySize"] == 1).astype(int)

In [ ]:
# Encoding Categorical Variables 

test_data["Sex"] = test_data["Sex"].map({"male": 0, "female": 1})

In [ ]:
test_data = pd.get_dummies(test_data, columns=["AgeGroup", "Embarked"], drop_first=True, dtype=int)

In [ ]:
# Scaling Numerical Features

test_data["Fare"] = scaler.fit_transform(test_data[["Fare"]])

In [ ]:
# Handling the name feature 

test_data["Title"] = test_data["Name"].str.extract(" ([A-Za-z]+)\.", expand=False)

In [ ]:
test_data["Title"].value_counts()

In [ ]:

test_data["Title"] = test_data["Title"].replace(["Lady", "Countess", "Capt", "Col", "Don", "Dr", "Major", "Rev", "Sir", "Jonkheer", "Dona"], "Rare")
test_data["Title"] = test_data["Title"].replace(["Mlle", "Ms"], "Miss")
test_data["Title"] = test_data["Title"].replace("Mme", "Mrs")

In [ ]:
test_data  = pd.get_dummies(test_data, columns=["Title"], drop_first=True, dtype=int)

In [ ]:
test_data = test_data.drop(["Name", "Ticket"], axis=1)

In [ ]:
test_data.head()

In [ ]:
test_predictions = optimized_model.predict(test_data)

In [ ]:
# Creating a DataFrame with PassengerId and the predicted Survived labels

submission = pd.DataFrame({
    "PassengerId": test_data["PassengerId"],
    "Survived": test_predictions
})

submission.to_csv("submission.csv", index=False)

In [ ]:
submission